In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *

"""
First model: parabola with a focal point at 1/(4a)
Only one "half XX"
Parameters:
- parabola coefficient A
- parabola Y axis translation
- parabola Z axis rotation

Notes:
- Parabola origin will need to account for rotation (bit less than 1000)
"""

cylinder = tlm.ImplicitCylinder(*torch.tensor([-50/2, 50/2, 37.02/2], dtype=torch.float64).unbind())

# Parameters
A = tlm.parameter(-2.0)
T = tlm.parameter(100.0)
#R = tlm.parameter(4.3)
R = tlm.parameter(4.3)

# Primary mirror
primary = tlm.Parabola(2000, A=A, normalize=True)

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    XXLightSource.load(half=True),
    #RaysViewerPlane(2000, "input"),
    tlm.Gap(990),
    tlm.Translate3D(y=T),
    tlm.Rotate3D(z=R),
    tlm.ReflectiveSurface(primary),
    Focus(primary),
    tlm.FocalPoint(),
    BoxLoss(450, 1.0, 0.1),
    #NonImagingRod(cylinder),
)

print(list(optics.named_parameters()))
print(R is optics[4].z)

xxrender(optics)


In [ ]:
sampling = {"xx": 2000}

#gridsearch1d(optics, T, torch.linspace(-50, 250, 60), sampling)
#gridsearch1d(optics, R, torch.linspace(-10, 10, 50), sampling)

In [ ]:
param_groups = [
    {'params': [A], 'lr': 1e-4},
    #{'params': [T], 'lr': 20},
    #{'params': [R], 'lr': 1e-2},
]

record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 50, "disable_viewer": True},
    dim = 3,
    num_iter = 10000
)


plot_record(record, param_groups, optics)
record.best()

print("A", A.item())
F = 1./(primary._sag.unnorm(primary.diameter / 2) * 4)
print("F", F.item())
print("T", T.item())
print("R", R.item())

xxrender(optics)



In [ ]:
import matplotlib.pyplot as plt

Y = torch.linspace(-600, 600, 500)

bounds = XXBounds(margin=450, scale=1, beta=0.1)

plt.figure()
plt.plot(Y.tolist(), bounds(Y).tolist())
#plt.gca().set_ylim([0, 100])
plt.gca().set_xlim([-600, 600])
